# Running a series of simulations

This guide explains how to run multiple simulations using the **CrowdMechanics** library and how to manage their outputs.

## Steps

1. **Run simulations:**  
   The script will execute several calls to the CrowdMechanics library, each representing a different simulation scenario or time step.

2. **Save outputs:**  
   Simulation results are automatically saved in the `outputXML/` folder.

3. **File naming:**  
   Each output file follows this pattern:  
   ```
   AgentDynamics output t=TIME_VALUE.xml
   ```
   Replace `TIME_VALUE` with the corresponding simulation time or identifier.

4. **Export to ChAOS:**  
   The output files can be easily converted into `.csv` files structured for direct import into [ChAOS](https://project.inria.fr/crowdscience/project/ocsr/chaos/). ChAOS uses these CSV files—containing time, x, and y coordinates for each agent—to visualize and animate agent trajectories for further analysis and video generation.

## Simulation

In [ ]:
import ctypes
from pathlib import Path
from shutil import copyfile
import xml.etree.ElementTree as ET

### Parameters
dt = 0.1  # The "TimeStep" in Parameters.xml
Ndt = 10  # The number of successive calls to the library

outputPath = "outputXML/"
inputPath = "inputXML/"
Path(outputPath).mkdir(parents=True, exist_ok=True)
Path(inputPath).mkdir(parents=True, exist_ok=True)

# Load the library into ctypes
Clibrary = ctypes.CDLL("../../src/mechanical_layer/build/libCrowdMechanics.dylib")  # Use .so for Linux and .dylib for MacOS
# The file name for the dynamical quantities will be used to build the names of the output files
agentDynamicsFilename = "AgentDynamics.xml"
# Prepare the call to CrowdMechanics
files = [b"Parameters.xml", b"Materials.xml", b"Geometry.xml", b"Agents.xml", agentDynamicsFilename.encode("ascii")]
nFiles = len(files)
filesInput = (ctypes.c_char_p * nFiles)()
filesInput[:] = files


### Actual loop
for t in range(Ndt):
    print("Looping the Crowd mechanics engine - t=%.1fs..." % (t * dt))
    # Copy Agent dynamics input file
    copyfile("dynamic/" + agentDynamicsFilename, inputPath + rf"AgentDynamics input t={t * dt:.1f}.xml")
    # Call the mechanical layer
    Clibrary.CrowdMechanics(filesInput)
    # Copy Agent dynamics output file to the directory that will be read by ChAOS
    copyfile("dynamic/" + agentDynamicsFilename, outputPath + rf"AgentDynamics output t={(t + 1) * dt:.1f}.xml")
    # Save the AgentInteractions file, if it exists
    try:
        copyfile("dynamic/AgentInteractions.xml", outputPath + rf"AgentInteractions t={(t + 1) * dt:.1f}.xml")
    except FileNotFoundError:
        pass
    # Prepare next run: Add dynamics tag to the input file
    # This is the step where you decide the Fp and Mp that will drive the next dt seconds for each agent
    XMLtree = ET.parse("dynamic/" + agentDynamicsFilename)
    agentsTree = XMLtree.getroot()

    """ Sample dummy code where we put the same constant values for each agent (can be replaced by your own decisional layer code)
        for agent in agentsTree:
            dynamicsItem = ET.SubElement(agent, "Dynamics")
            dynamicsItem.attrib["Fp"] = "100,100"
            dynamicsItem.attrib["Mp"] = "10"
        To be replaced...
    """

    XMLtree.write("dynamic/" + agentDynamicsFilename)

# Done!
print(f"Loop terminated at t={Ndt * dt:.1f}!")

Looping the Crowd mechanics engine - t=0.0s...
Looping the Crowd mechanics engine - t=0.1s...
Looping the Crowd mechanics engine - t=0.2s...
Looping the Crowd mechanics engine - t=0.3s...
Looping the Crowd mechanics engine - t=0.4s...
Looping the Crowd mechanics engine - t=0.5s...
Looping the Crowd mechanics engine - t=0.6s...
Looping the Crowd mechanics engine - t=0.7s...
Looping the Crowd mechanics engine - t=0.8s...
Looping the Crowd mechanics engine - t=0.9s...
Loop terminated at t=1.0!


Error: no Dynamics tag present for agent 0
Error: no Dynamics tag present for agent 0
Error: no Dynamics tag present for agent 0
Error: no Dynamics tag present for agent 0
Error: no Dynamics tag present for agent 0
Error: no Dynamics tag present for agent 0
Error: no Dynamics tag present for agent 0
Error: no Dynamics tag present for agent 0
Error: no Dynamics tag present for agent 0
Error: no Dynamics tag present for agent 0


## Export to ChAOS

In [2]:
from configuration.backup import xml_to_Chaos

filenameCSV = "all_trajectories.csv"
PathXML = Path("outputXML").resolve()
PathCSV = Path("outputCSV").resolve()
PathCSV.mkdir(parents=True, exist_ok=True)

xml_to_Chaos.export_dict_to_CSV(PathCSV, PathXML)
xml_to_Chaos.export_from_CSV_to_CHAOS(PathCSV, dt)

Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/Shape/tutorials/mechanical_layer/outputXML/AgentDynamics output t=0.1.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/Shape/tutorials/mechanical_layer/outputXML/AgentDynamics output t=0.2.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/Shape/tutorials/mechanical_layer/outputXML/AgentDynamics output t=0.3.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/Shape/tutorials/mechanical_layer/outputXML/AgentDynamics output t=0.4.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/Shape/tutorials/mechanical_layer/outputXML/AgentDynamics output t=0.5.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/Shape/tutorials/mechanical_layer/outputXML/AgentDynamics output t=0.6.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/Shape/tutorials/m